In [3]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project'

In [4]:
import sys, os
import numpy as np
%aimport data
%aimport register

In [15]:
case = data.Emory4DCTCase(data_root='data/Emory-4DCT', case_name='Case1Pack', phases=[0, 50])
case.load_niftis()
case

Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz


Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', 2 phases)

In [18]:
case.view(groupby=['phase', 'y'], x='x', y='z')

:DynamicMap   [phase,y]
   :Image   [x,z]   (case1)

In [20]:
case.array

<xarray.DataArray 'case1' (phase: 2, x: 256, y: 256, z: 94)>
array([[[[ 37.,  37.,  40., ...,  34.,  28.,  16.],
         [ 37.,  26.,  47., ...,  23.,  21.,  17.],
         [ 46.,  31.,  20., ...,  21.,  33.,  20.],
         ...,
         [580., 584., 603., ..., 284., 360., 305.],
         [308., 298., 324., ..., 153., 137., 169.],
         [185., 177., 188., ..., 115., 103., 125.]],

        [[ 38.,  32.,  23., ...,  36.,  25.,  12.],
         [ 25.,  31.,  36., ...,  26.,  26.,  13.],
         [ 23.,  34.,  46., ...,  21.,  22.,  24.],
         ...,
         [591., 587., 615., ..., 300., 356., 297.],
         [316., 297., 311., ..., 166., 148., 152.],
         [190., 184., 184., ..., 125., 111., 115.]],

        [[ 28.,  33.,  29., ...,  35.,   9.,  31.],
         [ 33.,  35.,  16., ...,  34.,  26.,  21.],
         [ 29.,  27.,  44., ...,  41.,  23.,  15.],
         ...,
...
         ...,
         [298., 304., 330., ..., 149., 211., 158.],
         [172., 167., 177., ..., 126., 123., 140.],
         [133., 133., 138., ...,  97., 121., 120.]],

        [[ 30.,  46.,  39., ...,  19.,  27.,  25.],
         [ 29.,  36.,  26., ...,  20.,  30.,  29.],
         [ 26.,  34.,  23., ...,  18.,  32.,  17.],
         ...,
         [321., 304., 317., ..., 161., 204., 143.],
         [172., 162., 184., ..., 132., 106., 124.],
         [143., 140., 140., ..., 119., 119., 111.]],

        [[ 30.,  35.,  31., ...,  26.,  30.,  27.],
         [ 30.,  31.,  27., ...,  13.,  27.,  32.],
         [ 37.,  36.,  31., ...,  21.,  17.,  26.],
         ...,
         [314., 309., 312., ..., 161., 196., 150.],
         [177., 162., 172., ..., 135., 105., 122.],
         [153., 146., 147., ..., 126., 112., 118.]]]])
Coordinates:
  * phase    (phase) int64 0 50
  * x        (x) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * y        (y) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * z        (z) float64 0.0 2.5 5.0 7.5 10.0 ... 222.5 225.0 227.5 230.0 232.5

In [47]:
%autoreload
reg = register.Registration(
    fixed=case.array.sel(phase=0),
    moving=case.array.sel(phase=50),
    transform='rigid'
)
reg.execute()

Time spent on saving the results, applying the final transform etc.: 891 ms.
ELASTIX version: 5.000
Command line options from ElastixBase:
-fMask    unspecified, so no fixed mask used
-mMask    unspecified, so no moving mask used
-out      ./
-threads  unspecified, so all available threads are used
  The default value "true" is used instead.

  From elastix 4.8 it defaults to true!
This may change the behavior of your registrations considerably.

Command line options from TransformBase:
-t0       unspecified, so no initial transform used

Reading images...
Reading images took 0 ms.

  A default pyramid schedule is used.
  A default pyramid schedule is used.
  The default value "false" is used instead.
  The default value "GeometricalCenter" is used instead.
Transform parameters are initialized as: [0, 0, 0, 0, 0, 0]
Scales are estimated automatically.
Scales for transform parameters are: [10399.20103972063, 10399.201039720687, 10670.277079441297, 1, 1, 1]
Initialization of all componen

67	-1.751476	34.031790	2.040686	0.007013	2.2
68	-1.754183	33.974147	2.042826	0.006056	2.2
69	-1.759285	33.933726	2.044329	0.010613	2.4
70	-1.801057	34.834291	2.011355	0.016137	2.3
71	-1.735658	34.777154	2.013416	0.012159	2.4
72	-1.738710	34.826398	2.011640	0.007435	2.2
73	-1.754154	34.810164	2.012225	0.007546	2.1
74	-1.776083	34.891988	2.009279	0.009959	2.1
75	-1.760649	35.285884	1.995218	0.008792	2.1
76	-1.783160	35.903429	1.973565	0.011673	2.1
77	-1.776201	35.878810	1.974419	0.006792	2.2
78	-1.769907	36.674774	1.947170	0.010719	2.1
79	-1.761641	37.410102	1.922657	0.009368	2.2
80	-1.769635	37.382237	1.923575	0.011522	2.1
81	-1.773642	37.323009	1.925528	0.014524	2.2
82	-1.774354	37.309686	1.925968	0.011071	2.1
83	-1.793629	37.250201	1.927935	0.016345	2.2
84	-1.771208	37.194588	1.929777	0.010714	2.1
85	-1.773159	37.431861	1.921941	0.007041	2.2
86	-1.757506	37.392984	1.923221	0.010924	2.2
87	-1.794831	38.143134	1.898827	0.014319	2.2
88	-1.743342	38.148155	1.898666	0.009197	2.2
89	-1.7766

250	-1.753719	98.068719	0.943175	0.012630	3.5
251	-1.742729	99.064174	0.935355	0.016754	2.1
252	-1.757037	99.004688	0.935818	0.008532	2.1
253	-1.752705	99.231316	0.934055	0.013247	2.1
254	-1.764232	99.535941	0.931694	0.018140	2.1
255	-1.772338	99.476498	0.932154	0.012084	2.1
Time spent in resolution 0 (ITK initialization and iterating): 0.616 s.
Stopping condition: Maximum number of iterations has been reached.
Settings of AdaptiveStochasticGradientDescent in resolution 0:
( SP_a 112.302603 )
( SP_A 20.000000 )
( SP_alpha 1.000000 )
( SigmoidMax 1.000000 )
( SigmoidMin -0.059486 )
( SigmoidScale 0.000011 )


Resolution: 1
  The default value "false" is used instead.
  The default value "true" is used instead.
Setting the fixed masks took: 0 ms.
Setting the moving masks took: 0 ms.
  The default value "1" is used instead.
  The default value "false" is used instead.
  The default value "32" is used instead.
  The default value "32" is used instead.
  The default value "32" is used inste

65	-1.554913	46.486129	2.765955	0.029938	2.5
66	-1.539704	47.486129	2.725568	0.015892	2.2
67	-1.573777	47.179321	2.737833	0.018331	2.2
68	-1.548390	48.179312	2.698258	0.029697	2.2
69	-1.568077	47.872517	2.710277	0.043102	2.2
70	-1.548978	48.872517	2.671488	0.020230	2.2
71	-1.553598	49.872517	2.633794	0.015212	2.3
72	-1.545097	50.713642	2.602903	0.016457	2.2
73	-1.563533	51.620031	2.570415	0.022183	2.3
74	-1.583807	52.620026	2.535501	0.005578	2.2
75	-1.550381	52.331635	2.545472	0.005817	2.2
76	-1.561677	52.604272	2.536043	0.026980	2.2
77	-1.578455	52.315483	2.546033	0.007595	2.2
78	-1.544368	53.156292	2.517165	0.022797	2.3
79	-1.552770	53.001699	2.522423	0.039179	2.2
80	-1.544463	54.001699	2.488792	0.032574	2.2
81	-1.569825	55.001699	2.456045	0.012818	2.2
82	-1.557174	56.000154	2.424198	0.015558	2.2
83	-1.579483	56.993648	2.393318	0.017947	2.3
84	-1.544147	57.990038	2.363129	0.017390	2.2
85	-1.571584	58.261155	2.355045	0.012855	2.3
86	-1.566917	57.955931	2.364149	0.017604	2.3
87	-1.5345

245	-1.548255	125.451776	1.274574	0.023190	2.7
246	-1.535886	126.451776	1.265930	0.015811	2.3
247	-1.550865	126.144968	1.268570	0.010522	2.4
248	-1.572103	125.903407	1.270656	0.014444	2.8
249	-1.568961	125.784379	1.271686	0.015066	2.9
250	-1.553315	126.508237	1.265445	0.012863	2.8
251	-1.575206	127.506387	1.256940	0.016281	2.8
252	-1.586626	127.200360	1.259536	0.011461	2.7
253	-1.557664	128.193203	1.251154	0.018344	2.7
254	-1.560707	128.239976	1.250762	0.015225	2.7
255	-1.561752	129.224317	1.242566	0.023773	2.7
Time spent in resolution 1 (ITK initialization and iterating): 0.620 s.
Stopping condition: Maximum number of iterations has been reached.
Settings of AdaptiveStochasticGradientDescent in resolution 1:
( SP_a 186.663624 )
( SP_A 20.000000 )
( SP_alpha 1.000000 )
( SigmoidMax 1.000000 )
( SigmoidMin -0.306808 )
( SigmoidScale 0.000017 )


Resolution: 2
  The default value "false" is used instead.
  The default value "true" is used instead.
Setting the fixed masks took: 0 ms.
Sett

139	-1.208549	53.843195	1.564011	0.031407	3.7
140	-1.224811	54.843189	1.543389	0.016224	2.2
141	-1.221785	55.768364	1.524789	0.018879	2.2
142	-1.225935	55.602938	1.528082	0.028215	2.1
143	-1.226323	55.123157	1.537713	0.024218	2.2
144	-1.198517	56.122371	1.517790	0.019634	2.1
145	-1.214766	57.121820	1.498372	0.016159	2.2
146	-1.240892	57.982147	1.482051	0.019657	2.2
147	-1.230920	57.565769	1.489905	0.029170	2.3
148	-1.224161	57.464238	1.491833	0.015337	2.1
149	-1.234061	58.335734	1.475445	0.017185	2.2
150	-1.232205	58.000342	1.481709	0.013385	2.1
151	-1.222828	57.842341	1.484679	0.009584	2.2
152	-1.207938	57.467813	1.491765	0.012186	2.2
153	-1.206677	57.494295	1.491262	0.014651	2.2
154	-1.228963	57.882187	1.483929	0.038435	2.3
155	-1.204916	58.881863	1.465358	0.016136	2.1
156	-1.222360	59.881770	1.447243	0.013468	2.2
157	-1.227531	59.607627	1.452165	0.022631	2.2
158	-1.207184	59.834021	1.448098	0.024096	2.1
159	-1.220460	60.807610	1.430864	0.026414	2.0
160	-1.186458	60.375119	1.438469	0

40	-0.729091	16.526273	6.469693	0.007187	2.8
41	-0.728281	17.456667	6.313170	0.018764	2.9
42	-0.727317	17.055281	6.379758	0.021348	2.0
43	-0.747494	16.653073	6.447906	0.009177	2.1
44	-0.732628	17.653063	6.281093	0.013544	2.1
45	-0.737332	18.518412	6.143553	0.015314	2.1
46	-0.723263	19.488965	5.996287	0.007313	2.1
47	-0.747587	20.485978	5.852181	0.006534	2.2
48	-0.757433	21.355330	5.732064	0.017433	2.1
49	-0.728548	22.265456	5.611485	0.009417	2.1
50	-0.724829	21.863346	5.664128	0.013621	2.1
51	-0.718284	22.715415	5.553727	0.009611	2.1
52	-0.746663	23.499232	5.455902	0.014728	2.2
53	-0.735256	23.136286	5.500768	0.014509	2.2
54	-0.738145	22.734077	5.551357	0.006366	2.2
55	-0.730716	22.455939	5.586888	0.019023	2.2
56	-0.721936	22.097469	5.633358	0.008102	2.2
57	-0.738684	22.798874	5.543144	0.011315	2.2
58	-0.735331	23.773317	5.422504	0.014784	2.2
59	-0.735185	23.778036	5.421932	0.015087	2.2
60	-0.721338	24.778000	5.303497	0.005969	2.2
61	-0.736341	24.391958	5.348601	0.008898	2.2
62	-0.7376

222	-0.747506	83.074951	2.332775	0.011096	3.5
223	-0.716507	83.765804	2.317392	0.016878	2.0
224	-0.744514	84.765430	2.295490	0.010843	2.0
225	-0.719017	85.757191	2.274165	0.007507	2.0
226	-0.718205	85.922839	2.270642	0.008902	2.0
227	-0.729400	86.672572	2.254831	0.019020	2.1
228	-0.728984	86.272387	2.263243	0.013130	2.1
229	-0.736849	87.272383	2.242340	0.007443	2.0
230	-0.739774	86.893669	2.250211	0.009966	2.1
231	-0.728293	86.788417	2.252408	0.017207	2.2
232	-0.718866	86.429675	2.259929	0.009258	2.2
233	-0.734022	87.423449	2.239216	0.017453	2.2
234	-0.725434	87.344567	2.240846	0.007781	2.1
235	-0.734585	86.946291	2.249114	0.006441	2.2
236	-0.746323	86.752863	2.253151	0.015288	2.2
237	-0.731322	86.410007	2.260343	0.014510	2.2
238	-0.731303	87.409892	2.239496	0.009198	2.2
239	-0.722055	87.007935	2.247830	0.008426	2.2
240	-0.735987	87.986608	2.227645	0.012532	2.2
241	-0.753798	87.614962	2.235267	0.012121	2.1
242	-0.728216	87.253588	2.242729	0.012225	2.2
243	-0.743898	88.240226	2.222473	0

In [56]:
result = reg.get_result()
result

<xarray.DataArray 'result' (x: 256, y: 256, z: 94)>
array([[[ 11.837539,  26.622408,  37.72808 , ...,  24.833061,
          30.184887,  16.082504],
        [ 36.160973,  30.380777,  33.433434, ...,  31.379728,
          17.48921 ,  31.64266 ],
        [ 49.680046,  39.727562,  34.634834, ...,  22.1264  ,
          24.536158,  36.95737 ],
        ...,
        [514.4498  , 539.215   , 554.64825 , ..., 278.64996 ,
         341.73624 , 286.20453 ],
        [243.47083 , 267.06705 , 250.61461 , ..., 143.83092 ,
         141.7168  , 162.49623 ],
        [194.4752  , 199.71593 , 198.32227 , ..., 114.27403 ,
         111.24799 , 118.08197 ]],

       [[ 20.339354,  40.90361 ,  40.598392, ...,  20.356924,
          30.36191 ,   8.108449],
        [  9.963987,  24.08935 ,  54.558033, ...,  23.004951,
          30.90548 ,  10.43718 ],
        [ 31.964005,  19.12292 ,  39.501507, ...,  24.36393 ,
          23.74886 ,  35.442272],
...
        [255.68192 , 239.9149  , 251.42438 , ..., 162.31924 ,
         190.76025 , 142.96272 ],
        [155.72003 , 151.11134 , 159.1175  , ..., 125.63848 ,
         105.29122 , 122.18294 ],
        [146.82271 , 143.0914  , 144.81146 , ..., 118.079994,
         117.8354  , 111.61432 ]],

       [[ 29.951647,  35.436264,  30.368181, ...,  25.30945 ,
          30.134869,  27.16029 ],
        [ 32.24763 ,  32.066772,  27.385168, ...,  11.473455,
          27.525118,  32.03229 ],
        [ 34.512245,  36.026688,  28.516693, ...,  21.642939,
          17.125156,  25.155457],
        ...,
        [253.21852 , 242.52145 , 247.60599 , ..., 162.09148 ,
         185.01788 , 147.84532 ],
        [160.93129 , 150.45244 , 155.8478  , ..., 128.38765 ,
         102.67261 , 119.774345],
        [153.11273 , 145.53102 , 147.27466 , ..., 125.50419 ,
         111.21716 , 117.32645 ]]], dtype=float32)
Coordinates:
  * x        (x) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * y        (y) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * z        (z) float64 0.0 2.5 5.0 7.5 10.0 ... 222.5 225.0 227.5 230.0 232.5

In [59]:
result.hvplot(groupby=['y'], x='x', y='z', cmap='greys_r', frame_width=500, data_aspect=1)

:DynamicMap   [y]
   :Image   [x,z]   (result)